In [1]:
!pip install pandas

     |████████████████████████████████| 11.5 MB 4.0 MB/s            
     |████████████████████████████████| 15.7 MB 5.2 MB/s            


In [2]:
import pandas as pd
print(pd.__version__)

1.3.4


## Seriesの作成
nameはOptional

In [3]:
s = pd.Series([1, 2, 3], name="s")
s

0    1
1    2
2    3
Name: s, dtype: int64

# Dataframe作成

In [4]:
df = pd.DataFrame({
    "A": ["a", "b", "a"],
    "B": [1, 3, 5],
    "C": [10, 11, 12],
    "D": [2, 4, 6]
})
df

,A,B,C,D
0,a,1,10,2
1,b,3,11,4
2,a,5,12,6


# 列選択

In [5]:
df["A"]

0    a
1    b
2    a
Name: A, dtype: object

In [6]:
df[["A", "B"]]

,A,B
0,a,1
1,b,3
2,a,5


## 条件に応じた列選択

In [7]:
df.loc[:, [c.startswith("A") for c in df.columns]]
df.loc[:, df.columns.str.startswith("A")]

,A
0,a
1,b
2,a


In [8]:
## 列の入れ替え

In [9]:
df[["B", "A"]]

,B,A
0,1,a
1,3,b
2,5,a


## 列の追加

In [10]:
df["E"] = df["B"] * 2
df["F"] = df["B"].map(lambda x: x * 2)
df = df.assign(G = lambda df: df.B * 2)
df

,A,B,C,D,E,F,G
0,a,1,10,2,2,2,2
1,b,3,11,4,6,6,6
2,a,5,12,6,10,10,10


## 条件による行選択

### 単独条件

In [11]:
df.loc[df["B"] <= 4]
df.query("B <= 4")

,A,B,C,D,E,F,G
0,a,1,10,2,2,2,2
1,b,3,11,4,6,6,6


複数条件

In [12]:
df.loc[(df["B"] == 1) | (df["C"] == 12)]
df.query("B == 1 | C == 12")

,A,B,C,D,E,F,G
0,a,1,10,2,2,2,2
2,a,5,12,6,10,10,10


### 含まれているかなどの演算

In [13]:
l = [1, 3]
df.query("B in @l")

,A,B,C,D,E,F,G
0,a,1,10,2,2,2,2
1,b,3,11,4,6,6,6


## 重複行の抽出

In [14]:
df.loc[df.duplicated()]

,A,B,C,D,E,F,G


## 重複行の削除


In [15]:
df.loc[3] = df.loc[2]
df

,A,B,C,D,E,F,G
0,a,1,10,2,2,2,2
1,b,3,11,4,6,6,6
2,a,5,12,6,10,10,10
3,a,5,12,6,10,10,10


In [16]:
df.drop_duplicates()

,A,B,C,D,E,F,G
0,a,1,10,2,2,2,2
1,b,3,11,4,6,6,6
2,a,5,12,6,10,10,10


## numpyへの変換

In [17]:
df.values

array([['a', 1, 10, 2, 2, 2, 2],
       ['b', 3, 11, 4, 6, 6, 6],
       ['a', 5, 12, 6, 10, 10, 10],
       ['a', 5, 12, 6, 10, 10, 10]], dtype=object)

## Groupby

In [18]:
dates = [
"2020-08-21",
"2020-08-21",
"2020-08-22",
"2020-08-23",
"2020-08-22",
]

temp = [20, 10, 7, 9, 1]
rain = [0.2, 0.1, 0.3, 0.1, 0.01]

d = dict(
    date=dates,
    temp=temp,
    rain=rain
)

df = pd.DataFrame.from_dict(d)

### build-inの演算

In [19]:
df.groupby("date").var()

,temp,rain
date,,
2020-08-21,50.0,0.00500
2020-08-22,18.0,0.04205
2020-08-23,NaN,NaN


In [20]:
df.groupby("date")[["temp"]].sum()

,temp
date,
2020-08-21,30
2020-08-22,8
2020-08-23,9


In [21]:
import numpy as np

In [22]:
# 複数の演算をまとめて
df.groupby("date").agg({"temp": [np.mean, np.var], "rain": [np.std]})

temp            rain
            mean   var       std
date                            
2020-08-21  15.0  50.0  0.070711
2020-08-22   4.0  18.0  0.205061
2020-08-23   9.0   NaN       NaN

### 任意の演算

In [23]:
df.groupby("date").agg(lambda x: print(x))

0    20
1    10
Name: temp, dtype: int64
2    7
4    1
Name: temp, dtype: int64
3    9
Name: temp, dtype: int64
0    0.2
1    0.1
Name: rain, dtype: float64
2    0.30
4    0.01
Name: rain, dtype: float64
3    0.1
Name: rain, dtype: float64


,temp,rain
date,,
2020-08-21,None,None
2020-08-22,None,None
2020-08-23,None,None


In [24]:
df.groupby("date").apply(lambda x: print(x))

         date  temp  rain
0  2020-08-21    20   0.2
1  2020-08-21    10   0.1
         date  temp  rain
2  2020-08-22     7  0.30
4  2020-08-22     1  0.01
         date  temp  rain
3  2020-08-23     9   0.1


""


## vstack, hstack

In [25]:
df1 = pd.DataFrame({"A": [1, 2, 3], "B": [2, 3, 4]})
df1_t = pd.DataFrame({"A": [4, 5, 6], "B": [5, 6, 7]})
df2 = pd.DataFrame({"C": ["a", "b", "c"], "D": [0.1, 0.2, 0.3]})
s1 = pd.Series([10, 11, 12], name="s1")
s2 = pd.Series(["ABC", "NMK", "XYZ"], name="s2")

In [26]:
# hstack
pd.concat([df1, s1, s2], axis=1)

,A,B,s1,s2
0,1,2,10,ABC
1,2,3,11,NMK
2,3,4,12,XYZ


In [27]:
pd.concat([df1, df2], axis=1)

,A,B,C,D
0,1,2,a,0.1
1,2,3,b,0.2
2,3,4,c,0.3


In [28]:
# vstack
pd.concat([df1, df1_t])

,A,B
0,1,2
1,2,3
2,3,4
0,4,5
1,5,6
2,6,7


In [29]:
pd.concat((df1, df2))

,A,B,C,D
0,1.0,2.0,NaN,NaN
1,2.0,3.0,NaN,NaN
2,3.0,4.0,NaN,NaN
0,NaN,NaN,a,0.1
1,NaN,NaN,b,0.2
2,NaN,NaN,c,0.3


In [32]:
df1 = pd.DataFrame({
    "Fruit": ["Apple", "Banana", "Pear"],
    "Origin": ["America", "Hawai", "Italy"],
    "Phosphorus (mg/100g)": [11, 22, 12]
})

df2 = pd.DataFrame({
    "Name": ["Apple", "Banana", "Pear"],
    "Origin": ["France", "Hawai", "Italy"],
    "Potassium (mg/100g)": [107, 358, 115]})

In [38]:
pd.merge(
    df1, df2, 
    left_on="Fruit", right_on="Name", 
how="inner")

,Fruit,Origin_x,Phosphorus (mg/100g),Name,Origin_y,Potassium (mg/100g)
0,Apple,America,11,Apple,France,107
1,Banana,Hawai,22,Banana,Hawai,358
2,Pear,Italy,12,Pear,Italy,115


In [40]:
pd.merge(
    df1, df2, 
    left_on=["Fruit", "Origin"], right_on=["Name", "Origin"], 
    how="outer"
)

,Fruit,Origin,Phosphorus (mg/100g),Name,Potassium (mg/100g)
0,Apple,America,11.0,NaN,NaN
1,Banana,Hawai,22.0,Banana,358.0
2,Pear,Italy,12.0,Pear,115.0
3,NaN,France,NaN,Apple,107.0


In [41]:
pd.merge(
    df1, df2, 
    left_on="Origin", right_on="Origin", 
    how="left"
)

,Fruit,Origin,Phosphorus (mg/100g),Name,Potassium (mg/100g)
0,Apple,America,11,NaN,NaN
1,Banana,Hawai,22,Banana,358.0
2,Pear,Italy,12,Pear,115.0
